In [1]:
import gmql as gl

In [2]:
# path of the dataset
input_path = "/home/luca/Documenti/GMQL-Python/resources/hg_narrowPeaks_short/"
np_parser = gl.parsers.NarrowPeakParser()
dataset = gl.GMQLDataset(parser=np_parser)
dataset = dataset.load_from_path(path=input_path)

2017-04-02 12:59:15,348 - gmql_logger - INFO - loading metadata
2017-04-02 12:59:17,700 - gmql_logger - INFO - parsing metadata
2017-04-02 12:59:17,704 - gmql_logger - INFO - collecting metadata
2017-04-02 12:59:23,059 - gmql_logger - INFO - dataframe construction


100%|██████████| 109/109 [00:03<00:00, 28.04it/s]


2017-04-02 12:59:32,456 - gmql_logger - INFO - loading region data
2017-04-02 12:59:32,912 - gmql_logger - INFO - parsing region data


/home/luca/spark-2.1.0-bin-hadoop2.7/python/pyspark/sql/session.py:336: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [ ]:
filtered = dataset.reg_select(lambda row: row['peak'] != -1 and row['chr'] == 'chr6')

In [6]:
df = dataset.meta_dataset.toPandas()

In [16]:
m = dataset.meta_dataset

In [36]:
import pyspark.sql.functions as functions

In [39]:
f = m.filter(functions.array_contains(m['antibody'], 'CTCF'))

In [46]:
row = f.rdd.take(1)[0]

In [48]:
row.asDict()

{'ID': ['730'],
 'antibody': ['CTCF'],
 'antibody_antibodyDescription': ['Rabbit polyclonal. Antibody Target: CTCF'],
 'antibody_lab': ['Myers, Hardison, Snyder'],
 'antibody_label': ['CTCF (07-729)'],
 'antibody_lots': ['1350637 DAM1472197'],
 'antibody_orderUrl': ['http://www.millipore.com/catalogue/item/07-729'],
 'antibody_tag': ['CTCF'],
 'antibody_target': ['CTCF'],
 'antibody_targetClass': ['TFSS'],
 'antibody_targetDescription': ['CTCF zinc finger transcription factor. A sequence specific DNA binding protein that functions as an insulator, blocking enhancer activity. It has also been suggested to block the spreading of chromatin structure in certain instances.'],
 'antibody_targetId': ['GeneCard:CTCF'],
 'antibody_targetUrl': ['http://www.genecards.org/cgi-bin/carddisp.pl?gene=CTCF'],
 'antibody_type': ['Antibody'],
 'antibody_validation': ['Human_-_CTCF_(07-729)(Western_blot,Motif_Enrichment):human_CTCF_validation_Myers.pdf Human_-_CTCF_(07-729)(Western_blot):human_CTCF_07-729